# Criação de Modelo - Import bibliotecas

In [2]:
import os
import numpy as np
import albumentations
import monai
import pandas as pd
import matplotlib.pyplot as plt

from PIL import Image
from os import listdir
from sklearn.model_selection import GroupKFold
from torch.utils.data import Dataset

from monai.transforms import AddChannel, Compose, RandRotate90, Resize, ScaleIntensity, ToTensor, RandAffine, apply_transform

In [3]:
path_raw = '../data/raw'
path_data = '../data/processed/train-jpegs'
path_train = '../data/processed'

img_size = 128

# Dataset Pytorch

In [4]:
train_transforms = Compose([ScaleIntensity(), 
                            Resize((img_size, img_size, img_size)), 
                            RandAffine( 
                                      prob=0.5,
                                      translate_range=(5, 5, 5),
                                      rotate_range=(np.pi * 4, np.pi * 4, np.pi * 4),
                                      scale_range=(0.15, 0.15, 0.15),
                                      padding_mode='border'),
                            ToTensor()]
                          )

In [5]:
class PEDataset(Dataset):
   
    def __init__(self, df, transform=None, img_size = 128):
        
        self.transform = transform
        self.img_size = img_size
        self.df = df
        
        tr = pd.read_csv(path_train+'/train_apply.csv')
    
    def __len__(self):
        return len(self.df)
    
    
    def __getitem__(self, index):
        
        row = self.df.iloc[index]
        
        path_img = path_data + '/' + row['StudyInstanceUID'] + '/' + row['SeriesInstanceUID']
        imgs_array = sorted([ path_img + '/' + img_path for img_path in listdir(path_img) ])
        
        imgs_input = np.stack([ np.asarray(Image.open(img)) for img in img_array ]).transpose(3,0,1,2)
        
        if self.transform is not None:
            imgs_input = apply_transform(self.transform, imgs_input)
        
        return imgs_input
        

In [48]:
dataset = PEDataset(df, transform= train_transforms)

In [6]:
def binary_cross_entropy(y_pred, y_true):
    total_bce_loss = np.sum(-y_true * np.log(y_pred) - (1 - y_true) * np.log(1 - y_pred))